## This notebook is developed for PICES 2022

### CODAR

CODAR is a technology that allows the measurement of surface ocean current velocities at a distance using the Doppler shift of reflected radio waves. The principle of Bragg scattering dictates that virtually all of the radio signal received by a CODAR antenna is being reflected back from ocean waves of a particular wavelength. The reflected signal is Doppler-shifted to higher or lower frequencies, depending on whether the ocean wave is moving, respectively, toward or away from the antenna. In the absence of ocean currents, this Doppler shift gives a measurement of the wave spee. If ocean currents are present, however, the Doppler shift gives a measurement of the wave speed PLUS the speed of the ocean current towards or away from the antenna. Because the speed of deep-water waves is known for a given wavelength, the wave speed can be subtracted from the total measured speed, resulting in a value for the speed of the ocean current alone.

The strait of Georgia CODAR system used by ONC is a 25 MHz model. A radio frequency of 25 MHz corresponds to a radio wavelength of 12 meters, so the CODAR system is sensitive to ocean wavelengths of 6 meter. The equation used in calculating ocean current velocities from the Bragg-scattering signal assume that the ocean waves being measured are "deep-water" waves. In other words, their wavelengths are less than twice the water depth. As depth decreases beyong this point, the waves will increasingly take on the character of "shallow-water" waves. Thus that for a 25 MHz system, ocean currents measured in water depths of less than 3 meters should be viewd with suspiction.  


### CODAR stations and combiners

Each CODAR systems measures the radial velocities of ocean surface currents, toward and away from the station's attenna. Four stations (VCOL, VION, VGPT, VATK) in the Strait of Georgia  have been placed in locations such that their areas of coverage overlap considerably. In the overlapping regions, it is possible to combine the radial data from at least two stations into total ocean currents. 

In the strait of Georgia, four antennas have been gradually installed since 2011. 


Note that the range of high-frequency radio waves is greatly reduced over fresh water compared to the range over salty, highly-conductive water. For this reason, during the spring freshet (June 2018), when the surface water in the strait of Gerogia is much less sality, less coverage of total ocean current would be expected.


### Grid

The total files organize the data into grids with y,x and time dimensions, while the radials files use arrays for the current data, with dimensions of time and index, where index links the currents to their lat/lon sampling locations. The x,y dimensions are the grid centers relative to the grid origin.

The radial files from combiner stations will contain data from multiple radials stations; the platform_name variable will supply the station name / call sign (i.e. VION) for each time step. 


### Data description

MAT file data structures and definitions

This section describes the MAT file structure in detail. HFRProgs's README and code also documents these structures, so only relevant ones will be listed in detail.

**Totals**

1. U/V: surface current total velocities calculated from the Radials structure
2. LonLat,2 vector containing longitude and latitude pairs for each U/V value
3. Timestamp: date in number format.

### Data included: June and Nov. 2018 of total vectors in formats of *.mat 



In [1]:
import sys
!{sys.executable} -m pip install onc  # ONC python client libaray 
!{sys.executable} -m pip install basemap #plotting map 
!{sys.executable} -m pip install basemap-data-hires # higher resolution map
!{sys.executable} -m pip install hdf5storage


In [31]:
from hdf5storage import savemat
from hdf5storage import loadmat
import os
from datetime import timedelta  
import scipy.stats as stats
import matplotlib.mlab as mlab
from matplotlib.pyplot import cm
import matplotlib
import matplotlib.pyplot as plt
import numpy.matlib
import numpy.ma as ma
import numpy as np
import glob
from datetime import datetime
from mpl_toolkits.basemap import Basemap
import pandas as pd
from matplotlib.patches import Ellipse
from sklearn.decomposition import PCA
from onc.onc import ONC

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%matplotlib nbagg
cmap = matplotlib.cm.jet

In [3]:
#make date consistent between matlab and python
def matlab2datetime(matlab_datenum):
    day = datetime.fromordinal(int(matlab_datenum))
    dayfrac = timedelta(days=matlab_datenum%1) - timedelta(days = 366)
    return day + dayfrac

# SOG Map is generated with station names added.
def plotBasemap(Latmin,Latmax,Lonmin,Lonmax,Title):   
    
    M=Basemap(projection='merc', llcrnrlat=Latmin,urcrnrlat=Latmax,llcrnrlon=Lonmin,
              urcrnrlon=Lonmax,resolution='h',area_thresh = 0.01)    
    
    M.drawparallels(np.arange(48.85,49.37,0.1) ,labels=[1,0,0,0]);
    M.drawmeridians(np.arange(-123.8,-122.95,0.3),labels=[0,0,0,1]);
    M.drawmapboundary()
    M.fillcontinents(color='grey')
    
    
    plt.title(Title)
    Stationnames=['VION','VGPT','VATK','VCOL'] 
    for i,Istation in enumerate((Stationnames)):
        XO,YO=M(orig_lon[Istation],orig_lat[Istation])
        M.plot(XO,YO,'r^',markersize=10)
        plt.text(XO,YO,Istation,fontsize=14)
        plt.title(Title,fontsize=14)
    return M    

#shink total vectors by using log function to avoid neigbour vectors overlap     
def vector_log(U_raw,V_raw):
    SpeedL=np.log10(np.sqrt(U_raw**2+V_raw**2))
    Theta=np.arctan(V_raw/U_raw)
    UL=np.sign(U_raw)*SpeedL*np.abs(np.cos(Theta))
    VL=np.sign(V_raw)*SpeedL*np.abs(np.sin(Theta))
    return UL, VL    

In [4]:
# Longitude and latitude of the four radar sites for plotting sites
orig_lon={}
orig_lat={}

orig_lon['VION']=-123.2053833
orig_lon['VCOL']=-123.1718833
orig_lon['VGPT']=-123.291
orig_lon['VATK']=-123.2644333
orig_lon['VDIG']=-130.4254500
orig_lon['VRID']=-130.3346833

orig_lat['VATK']=49.3300667
orig_lat['VCOL']=49.01805
orig_lat['VGPT']=48.87365
orig_lat['VION']=49.2158667
orig_lat['VDIG']=54.2625333
orig_lat['VRID']=54.2346333

#basemap boundary
stationnames=['VION','VGPT','VATK','VCOL'] 
latmin = 48.85
latmax = 49.39
lonmin =-123.8
lonmax =-122.9

latpar_min=round(latmin*10)//10
latpar_max=round(latmax*10)//10
lonpar_min=round(lonmin*10)//10
lonpar_max=round(lonmax*10)//10

In [4]:
# Collect data from Ocean 3.0
month='Nov'
DownloadPath='/home/jupyter/PICES2022/Data/'+month+'2018_mat'

onc = ONC('',outPath=DownloadPath) # token here
beginDT='2018-11-01T00:00:00.000Z'
endDT='2018-11-01T03:00:00.000Z'
extention='mat'
dataProductCode='CODARQCSC'
deviceCategoryCode='OCEANOGRAPHICRADAR'
locationCode='SOGCS'


filters = {
    'locationCode': locationCode,
    'deviceCategoryCode': deviceCategoryCode,
    'dataProductCode': dataProductCode,
    'extension': extention,
    'dateFrom': beginDT,
    'dateTo': endDT,'dpo_IncludeRadials':'0'}

onc.orderDataProduct(filters, includeMetadataFile=False)

Request Id: 12359489
Estimated File Size: No estimated file size available.
Estimated Processing Time: No estimated processing time available.


   Running
   Running... working on time segment 1 of 1, for device deployment 1 of 1.
   Running... working on processing CODAR quality controlled data in deployment 1 of 1.
   Running... creation date on CODAR grid file STRAITOFGEORGIAARRAY_20160510T194413.000Z-CODAR.grd is 10-May-2016 19:44:13, in deployment 1 of 1............
   Running... working on time step 6 of 6, 2018-11-01 05:00:00, in deployment 1 of 1...
   Transferring (StraitofGeorgia_StraitofGeorgiaCODARSystem_OceanographicRadarSystem_20181101T000000.000Z_20181101T050000.000Z-Totals_Clean.mat) to the FTP server...........
Total run time: a moment
Total download Time: 0.164 seconds
1 files (1.5 MB) downloaded


{'downloadResults': [{'url': 'https://data.oceannetworks.ca/api/dataProductDelivery?method=download&token=91ce2738-1e0c-43d3-9a77-f1d26535ff87&dpRunId=27439594&index=1',
   'status': 'complete',
   'size': 1497530,
   'file': 'StraitofGeorgia_StraitofGeorgiaCODARSystem_OceanographicRadarSystem_20181101T000000.000Z_20181101T050000.000Z-Totals_Clean.mat',
   'index': '1',
   'downloaded': True,
   'requestCount': 31,
   'fileDownloadTime': 0.164}],
 'stats': {'runTime': 0.121,
  'downloadTime': 0.164,
  'requestCount': 32,
  'totalSize': 1497530}}

In [ ]:
# Load data that are downloaded through API
datafiles=sorted(glob.glob(DownloadPath+'/*.mat'))

ifile=0
datai=loadmat(datafiles[ifile])

In [48]:
# 2532 is the number of grid points, which is a fixed number
# num_hours is the time length of the data in the loaded mat file

num_hours=len(datai['Totals']['U'])
u=np.ones((2532,num_hours))
v=np.ones((2532,num_hours))
date=[]
for j in range(num_hours):
    dataj=datai['Totals'][j][0]
    #u and v 
    u[:,j]=np.squeeze(dataj['U'])
    v[:,j]=np.squeeze(dataj['V'])

    #date
    timestamp=dataj['TimeStamp'][0][0]+0.0000000001
    datej=str(matlab2datetime(np.float(timestamp)))[0:13]
    date=np.append(date,datej)
#lon and lats for SOG total grids, which is fixed   
lonlat=dataj['LonLat']

In [54]:
#make u and v into format of dataframe with function of pandas
lon=lonlat[:,0]
lat=lonlat[:,1]
num_hours=len(date)
u_df=pd.DataFrame(u,index=[lon,lat],columns=date)
v_df=pd.DataFrame(v,index=[lon,lat],columns=date)

In [55]:
u_df

,,2018-11-01 00,2018-11-01 01,2018-11-01 02,2018-11-01 03,2018-11-01 04,2018-11-01 05,2018-11-01 06,2018-11-01 07,2018-11-01 08,2018-11-01 09,...,2018-11-02 14,2018-11-02 15,2018-11-02 16,2018-11-02 17,2018-11-02 18,2018-11-02 19,2018-11-02 20,2018-11-02 21,2018-11-02 22,2018-11-02 23
-123.015064,48.788191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-123.001455,48.788148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-122.987846,48.788104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-122.919804,48.787858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-122.906195,48.787804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-123.741076,49.426560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-123.727292,49.426604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-123.713508,49.426645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-123.699723,49.426685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1) Temporal availability

Every hour has different number of grid points that has available data. Before any use of these data, it will be helpful to check the availability.

In [ ]:
# Plot available number of data grid points

In [ ]:
# Select one of the grid cell and then plot out the ocean current map
# For instance, plot the total currents that has maximum data coverage during that month


# 2) Spatial Availability 

Spatial availability=number of available hours/total hours of that month

# 3) spectral analysis

1. select one grid cell, which has maximum spatial data avaialability
2. rotary spectra analysis for u and v



In [1]:
#remember need to adjust nfft based on the length of the data



In [2]:
#plot rotary spectra, 



# 4) PCA analysis


Sources

https://scikit-learn.org/stable/auto_examples/cross_decomposition/plot_pcr_vs_pls.html#

https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html

Steps of calculating PCA: 

1. Subtract the mean of each value;
2. Calculate the Covariance Matrix: The covariance matrix is a square matrix donoting the covariance of elements with each other. The covariance of an element with itself is nothing but just its variance;
3. Compute the eigenvalues and eigenvectors:

    3.1 The eigenvectors of the covariance matrix we get are orthononal to each other and each vector represents a principle axis;    
    3.2 A higher eigenvalue corresponds to a higher variability. Hence the principal axis with the higher eigenvalue will be an axis captuting higher variability in the data;    
    3.3 Orthogonal means the vectors are mutually perpendicular to each other; 
    
4. Sort eigenvalues in descending order:

    4.1 Sort the eigenvalues in the descending order along with their corrsponding eigenvectors;    
    4.2 Each column in the eigen vector-matrix corresponds to a principal compoment, so arranging them in desciending order of their eigenvalue will automatically arrange the principal component in desciending order of their variability;    
    4.3 So the first column in the rearranged eigen vector-matrix will be a principle compionent that captures the highest variability.
    
5. Select a subset from the rearraged eigenvalue matrix

6. Transform the data. Transform the data by having a dot product between the transpose of the eigenvector subset and the transpose of the mean-centered data. 


PC actually is the direction with variance scaled. Direction can be decided by the eigenvectors of covariance. 

# 5) Save one grid cell data and then load into matlab to run t-tide 

# 6) Calculate and plot the tidal analysis for one grid/site [u,v]



In [3]:
# Load tidal analysis results

In [4]:
#######
# load major, minor and angle of ellipse